# INF-616 - Tarefa 3

Professor: Alexandre Ferreira -- melloferreira@ic.unicamp.br  
Monitor: Lucas David -- ra188972@students.ic.unicamp.br

Instituto de Computação - Unicamp  
2018

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline

np.random.seed(13)

In [ ]:
# O codigo abaixo ira baixar e carregar o conjunto `spambase`.
# Leia sobre este conjunto aqui: https://archive.ics.uci.edu/ml/datasets/spambase

target_names = ['not-spam', 'spam']
label_field = 'label'
attributes = ['word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d',
              'word_freq_our', 'word_freq_over', 'word_freq_remove', 'word_freq_internet',
              'word_freq_order', 'word_freq_mail', 'word_freq_receive', 'word_freq_will',
              'word_freq_people', 'word_freq_report', 'word_freq_addresses', 'word_freq_free',
              'word_freq_business', 'word_freq_email', 'word_freq_you', 'word_freq_credit',
              'word_freq_your', 'word_freq_font', 'word_freq_000', 'word_freq_money',
              'word_freq_hp', 'word_freq_hpl', 'word_freq_george', 'word_freq_650',
              'word_freq_lab', 'word_freq_labs', 'word_freq_telnet', 'word_freq_857',
              'word_freq_data', 'word_freq_415', 'word_freq_85', 'word_freq_technology',
              'word_freq_1999', 'word_freq_parts', 'word_freq_pm', 'word_freq_direct',
              'word_freq_cs', 'word_freq_meeting', 'word_freq_original', 'word_freq_project',
              'word_freq_re', 'word_freq_edu', 'word_freq_table', 'word_freq_conference',
              'char_freq_;', 'char_freq_(', 'char_freq_[', 'char_freq_!', 'char_freq_$',
              'char_freq_#', 'capital_run_length_average', 'capital_run_length_longest',
              'capital_run_length_total', label_field]
features = [a for a in attributes if a != 'label']

d = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data',
                header=None, names=attributes, delimiter=',')

In [ ]:
def describe(x, y, classes):
    """Descreve um conjunto de dados.
    
    :param x: np.ndarray, as amostras no conjunto.
    :param y: np.ndarray, os rotulos associados as amostras `x`.
    :param classes: list, uma lista com os nomes de cada classe. 
    """
    print('amostras:', x.shape[0])
    print('características:', x.shape[1])

    _, counts = np.unique(y, return_counts=True)
    print('frequência das classes:', dict(zip(classes, counts)))

def show_datasets(x, y, title):
    """Encontra um "embedding" de um conjunto que alinha as direções
       de maximiza separação das amostras com os eixos da base canônica.
    
    :param x: np.ndarray, as amostras do conjunto.
    :param y: np.ndarray, os rotulos associados as amostras `x`.
    :param title: str, o titulo do conjunto a ser exibido.
    """
    e = make_pipeline(
        StandardScaler(),
        PCA(n_components=2)
    ).fit_transform(x)

    _ = plt.figure(figsize=(12, 6))
    _ = plt.scatter(e[:, 0], e[:, 1], c=y)
    _ = plt.title(title)
    _ = plt.axis('off')

x_train, x_test, y_train, y_test = train_test_split(d[features], d['label'],
                                                    test_size=0.25,
                                                    random_state=1821)

# Seleciona 10% dos spams.
p, = np.where(y_train == 1)
np.random.shuffle(p)
p = p[:int(.1 * len(p))]
# Encontra os indices em treino que pertencem aos 10% selecionados.
p = np.in1d(np.arange(len(y_train)), p)
# Seleciona todos os nao-spam ou os 10% spams selecionados.
p = (y_train == 0) | p
# Filtra o conjunto de treino, mantendo somente os selecionados.
x_train, y_train = x_train[p], y_train[p]

print('Conjunto Spambase treino:')
describe(x_train, y_train, target_names)
print('Conjunto Spambase test:')
describe(x_test, y_test, target_names)
show_datasets(x_train, y_train, 'train')
show_datasets(x_test, y_test, 'test')

**Utilizando tudo o que você aprendeu até aqui (e.g. CV, otimização de hiper-parâmetros, regularização), defina e treine um estimador para este problema. Avalie seu estimador sobre o conjunto de teste.**

Dica: vários estimadores possuem regularizadores implementados internamente, expondo somente um parâmetro a ser ajustado. Exemplos: `KNeighborsClassifier`, `RandomForestClassifier`, `SVC`, `MLPClassifier`, `SGDClassifier`, `Perceptron`.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

model = Pipeline([
    ('scale', StandardScaler()),
    ('pca', PCA()),
    ('svc', SVC())
])
params = [
    {
        #'pca__n_components': range(1, len(features) + 1),
        'svc__C': [10**x for x in range(-3, 4)],
        'svc__kernel': ['linear', 'rbf'],
    }, {
        #'pca__n_components': range(1, len(features) + 1),
        'svc__C': [10**x for x in range(-3, 4)],
        'svc__kernel': ['poly'],
        'svc__degree': range(2,5)
    }
]

gridsearch = GridSearchCV(model, params, cv=2, n_jobs=4).fit(x_train, y_train)
print(f'Best Estimator: {gridsearch.best_params_}')
ŷ_test = gridsearch.best_estimator_.predict(x_test)
print(f'Precision = {metrics.precision_score(y_true = y_test, y_pred = ŷ_test):.3f}')
print(f'Recall = {metrics.recall_score(y_true = y_test, y_pred = ŷ_test):.3f}')
print(f'F1 = {metrics.f1_score(y_true = y_test, y_pred = ŷ_test):.3f}')

**Utilize uma ou mais técnicas da biblioteca imbalanced-learn e compare os resultados com os obtidos acima**

In [ ]:
!pip install imblearn

In [ ]:
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

In [ ]:
model = Pipeline([
    ('scale', StandardScaler()),
    ('pca', PCA()),
    ('svc', SVC())
])
params = [
    {
        #'pca__n_components': range(1, len(features) + 1),
        'svc__C': [10**x for x in range(-3, 4)],
        'svc__kernel': ['linear', 'rbf'],
    }, {
        #'pca__n_components': range(1, len(features) + 1),
        'svc__C': [10**x for x in range(-3, 4)],
        'svc__kernel': ['poly'],
        'svc__degree': range(2,5)
    }
]

for balancer_class in [RandomOverSampler, SMOTE, ADASYN]:
    print(f'Conjunto Spambase apos balanceamento {balancer_class.__name__}:')
    balancer = balancer_class(random_state=0)
    x_resampled_train, y_resampled_train = balancer.fit_sample(x_train, y_train)

    describe(x_resampled_train, y_resampled_train, target_names)
    show_datasets(x_resampled_train, y_resampled_train, 'resampled train')
    
    gridsearch = GridSearchCV(model, params, cv=2, n_jobs=4).fit(x_resampled_train, y_resampled_train)
    print(f'Best Estimator: {gridsearch.best_params_}')
    ŷ_test = gridsearch.best_estimator_.predict(x_test)
    print(f'Precision = {metrics.precision_score(y_true = y_test, y_pred = ŷ_test):.3f}')
    print(f'Recall = {metrics.recall_score(y_true = y_test, y_pred = ŷ_test):.3f}')
    print(f'F1 = {metrics.f1_score(y_true = y_test, y_pred = ŷ_test):.3f}')

In [ ]:
# Bonus: Use 'class_weight': ['balanced'] instead of balancing with fake data

model = Pipeline([
    ('scale', StandardScaler()),
    ('pca', PCA()),
    ('svc', SVC())
])
params = [
    {
        #'pca__n_components': range(1, len(features) + 1),
        'svc__class_weight': ['balanced'],
        'svc__C': [10**x for x in range(-3, 4)],
        'svc__kernel': ['linear', 'rbf'],
    }, {
        #'pca__n_components': range(1, len(features) + 1),
        'svc__class_weight': ['balanced'],
        'svc__C': [10**x for x in range(-3, 4)],
        'svc__kernel': ['poly'],
        'svc__degree': range(2,5)
    }
]

gridsearch = GridSearchCV(model, params, cv=2, n_jobs=4).fit(x_train, y_train)
print(f'Best Estimator: {gridsearch.best_params_}')
ŷ_test = gridsearch.best_estimator_.predict(x_test)
print(f'Precision = {metrics.precision_score(y_true = y_test, y_pred = ŷ_test):.3f}')
print(f'Recall = {metrics.recall_score(y_true = y_test, y_pred = ŷ_test):.3f}')
print(f'F1 = {metrics.f1_score(y_true = y_test, y_pred = ŷ_test):.3f}')